In [110]:
import boto3
import re
import urllib.request as req
import os

In [107]:
def get_public_ip():
    data = str(req.urlopen('http://checkip.dyndns.com/').read())
    
    return re.compile(r'Address: (\d+.\d+.\d+.\d+)').search(data).group(1)


def create_vpc(boto_session, cidr, name):
    ec2 = boto_session.client('ec2')
    vpc = ec2.create_vpc(CidrBlock = cidr, 
                         TagSpecifications=[{'ResourceType': 'vpc', 'Tags': [{'Key': 'Name', 'Value': name}]}])
    vpc_id = vpc['Vpc']['VpcId']
    waiter = ec2.get_waiter('vpc_available')
    waiter.wait(VpcIds=[vpc_id])
    #default security group, open internal traffic
    sg = ec2.describe_security_groups(Filters=[{'Name': 'vpc-id','Values': [vpc_id]}])
    sg_id = sg['SecurityGroups'][0]['GroupId']
    sg_tag = name + '-internal-securitygroup'
    int_sg_tag = ec2.create_tags(Resources=[sg_id], Tags=[{'Key': 'Name', 'Value': sg_tag}])
    #default route table
    rt = ec2.describe_route_tables(Filters=[{'Name': 'vpc-id','Values': [vpc_id]}])
    rt_id = rt['RouteTables'][0]['Associations'][0]['RouteTableId']
    rt_tag = name + '-route-table'
    rt_tbl_tag = ec2.create_tags(Resources=[rt_id], Tags=[{'Key': 'Name', 'Value': rt_tag}])
        
    return vpc, sg, rt 
    
def create_subnet(boto_session, vpc, cidr, az, name, public=False):
    ec2 = boto_session.client('ec2')
    subnet = ec2.create_subnet(VpcId = vpc['Vpc']['VpcId'],
                            CidrBlock = cidr,
                            AvailabilityZone = az,
                            TagSpecifications=[{'ResourceType': 'subnet', 'Tags': [{'Key': 'Name','Value': name}]}])
    #auto assign ip
    if public == True:
        pub_sub = ec2.modify_subnet_attribute(SubnetId = subnet['Subnet']['SubnetId'],
                                          MapPublicIpOnLaunch = {'Value': True})
                  
    return subnet

def create_internet_gateway(boto_session, vpc, name):
    ec2 = boto_session.client('ec2')
    ig = ec2.create_internet_gateway(TagSpecifications=[{'ResourceType': 'internet-gateway', 'Tags': [{'Key': 'Name','Value': name}]}])
    ig_attach = ec2.attach_internet_gateway(InternetGatewayId = ig['InternetGateway']['InternetGatewayId'],
                                     VpcId=vpc['Vpc']['VpcId'])

    return ig

def create_internet_route(boto_session, route_table, cidr, gateway):
    ec2 = boto_session.client('ec2')
    rt = ec2.create_route(RouteTableId = route_tbl['RouteTables'][0]['RouteTableId'],
                          DestinationCidrBlock = cidr,
                          GatewayId = gateway['InternetGateway']['InternetGatewayId'])
                                
    return rt

def create_db_subnet_group(boto_session, db_identifier, subnet_ids):
    rds = boto_session.client('rds')
    dbsubnet = rds.create_db_subnet_group(DBSubnetGroupDescription='{db} subnet group'.format(db=db_identifier),
                                          DBSubnetGroupName='{db}-subnetgroup'.format(db=db_identifier),
                                          SubnetIds=subnet_ids,
                                          Tags=[{'Key': 'Name','Value': '{db}-subnetgroup'.format(db=db_identifier)}])
    
    return dbsubnet

def create_external_ec2_security_group(boto_session, vpc, name):
    ec2 = boto_session.client('ec2')
    secgrp = ec2.create_security_group(GroupName = name,
                                   Description = 'basic security group for ec2',
                                   VpcId = vpc['Vpc']['VpcId'],
                                   TagSpecifications = [{'ResourceType': 'security-group', 
                                                         'Tags': [{'Key': 'Name','Value': name}]}])
    ip22 = get_public_ip() + '/32'
    ingress = ec2.authorize_security_group_ingress(GroupId=secgrp['GroupId'],
                             IpPermissions=[
                                 {'IpProtocol': 'tcp',
                                  'FromPort': 80,
                                  'ToPort': 80,
                                  'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                                 {'IpProtocol': 'tcp',
                                  'FromPort': 443,
                                  'ToPort': 443,
                                  'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                                 {'IpProtocol': 'tcp',
                                  'FromPort': 22,
                                  'ToPort': 22,
                                  'IpRanges': [{'CidrIp': ip22}]}
                             ])
    
    return secgrp


def create_key(boto_session, key_name, path):
    ec2 = boto_session.client('ec2')
    priv_key = ec2.create_key_pair(KeyName = key_name)
    key_file = open(path + key_name, "w")
    f = key_file.write(priv_key.key_material)
    key_file.close()
    

def create_mysqldb_security_group(boto_session, vpc, name):
    ec2 = boto_session.resource('ec2')
    secgrp = ec2.create_security_group(GroupName=name,
                                   Description='basic security group for mysql db',
                                   VpcId=vpc.id,
                                   TagSpecifications=[{'ResourceType': 'security-group', 'Tags': [{'Key': 'Name','Value': name}]}])
    secgrp.authorize_ingress(GroupId=secgrp.id,
                             IpPermissions=[
                                 {'IpProtocol': 'tcp',
                                  'FromPort': 3306,
                                  'ToPort': 3306,
                                  'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
                             ])
    return secgrp


def launch_amazon_linux2(boto_session, name, subnet_id, security_groups, key_name, 
                         itype = 't2.micro', ami = 'ami-05b622b5fa0269787'):
    ec2 = boto_session.client('ec2')
    instance = ec2.run_instances(ImageId = ami,
                     InstanceType = itype,
                     MaxCount = 1,
                     MinCount = 1,
                     SubnetId = subnet_id,
                     BlockDeviceMappings = [{'DeviceName': '/dev/xvda',
                                           'VirtualName': name + '-storage',
                                           'Ebs': {'DeleteOnTermination': True,
                                                   'VolumeSize': 30}
                                          }],
                     TagSpecifications = [{'ResourceType': 'instance', 
                                         'Tags': [{'Key': 'Name','Value': name}]}],
                     SecurityGroupIds = security_groups,
                     KeyName = key_name)
    waiter = ec2.get_waiter('instance_status_ok')
    waiter.wait(InstanceIds=[instance['Instances'][0]['InstanceId']])
                            
    return instance

In [8]:
bsm = boto3.Session(profile_name = 'mjn', region_name = 'us-west-2')

In [38]:
prj_vpc, internal_sg, route_tbl = create_vpc(bsm, '213.213.48.0/24', 'tripoli-vpc')

In [39]:
dbsubnet01 = create_subnet(bsm, prj_vpc, '213.213.48.32/27', 'us-west-2b', 'tripoli-dbsubnet-01')
dbsubnet02 = create_subnet(bsm, prj_vpc, '213.213.48.64/27', 'us-west-2c', 'tripoli-dbsubnet-02')
publicsubnet = create_subnet(bsm, prj_vpc, '213.213.48.96/27', 'us-west-2a', 'tripoli-publicsubnet', public = True)

In [51]:
intgateway = create_internet_gateway(bsm, prj_vpc, 'tripoli-ig')
int_route = create_internet_route(bsm, route_tbl, '0.0.0.0/0', intgateway)

In [85]:
ec2secgrp = create_external_ec2_security_group(bsm, prj_vpc, 'tripoli-ec2-securitygroup')

In [108]:
tripoli_ec2 = launch_amazon_linux2(bsm, 'tripoli-ec2', publicsubnet['Subnet']['SubnetId'], 
                                   [internal_sg['SecurityGroups'][0]['GroupId'], ec2secgrp['GroupId']],
                                  'tripoli_ec2_key')

In [63]:
dbsubnetgrp = create_db_subnet_group(bsm, 'tripoli-db', 
                                     [dbsubnet01['Subnet']['SubnetId'], dbsubnet02['Subnet']['SubnetId']])

In [65]:
rds = bsm.client('rds')

In [71]:
mysqldb_rds = rds.create_db_instance(DBName='tripoliwordpress',
                       DBInstanceIdentifier='tripoli-db',
                       AllocatedStorage=20,
                       DBInstanceClass='db.t2.micro',
                       Engine='mysql',
                       MasterUsername='#####',
                       MasterUserPassword='#####',
                       StorageType='gp2',
                       StorageEncrypted=False,
                       AutoMinorVersionUpgrade=True,
                       MultiAZ=False,
                       DBSubnetGroupName=dbsubnetgrp['DBSubnetGroup']['DBSubnetGroupName'],
                       PubliclyAccessible=False,
                       VpcSecurityGroupIds=[internal_sg['SecurityGroups'][0]['GroupId']],
                       Tags=[{'Key': 'Name', 'Value': 'tripoli-db'}])